# Generators, Coroutines and all the (other) things

Adapted from https://github.com/pymc-devs/pymc4/blob/master/notebooks/pymc4_design_guide.ipynb

**This guide is NOT meant to serve as an introduction to probabilistic programming. It is meant for users of pyMC3, switcing to pyMC4, explaining the new model creation and execution paradigm.**

With Theano, reaching the end of its life, pyMC core developers decided to port the framework's backend to Tensorflow Probability (TF Probability/ TPF)(Listen to Weicki's rationale here).  
But there are some fundamental design issues which the developers faced. In this document, we go over those problems, and their suggested solutions.

## The Problem With TF Probability

In [189]:
import tensorflow_probability as tfp
import tensorflow as tf
from tensorflow_probability import distributions as tfd
from pprint import pprint

### What do we want from a PPL ?

We want to implement a probabilisitc program (think of a Bayesian model that you usually represent in a directed acyclic graph) that does 2 things: 

1. Forward sampling to get prior (predictive) samples; 
2. Reverse evaluation on some inputs to compute the log-probability (if we conditioned on the observed, we are evaluating the unnormalized posterior distribution of the unknown random variables). 

Specifically for computing the log_prob, we need to keep track of the dependency. For example, if we have something simple like `x ~ Normal(0, 1), y ~ Normal(x, 1)`, in the reverse mode (goal 2 from above) we need to swap `x` with the input `x` (either from user or programmatically) to essentially do `log_prob = Normal(0, 1).log_prob(x_input) + Normal(x_input, 1).log_prob(y_input)`.

There are a few approaches to this problem. For example, in PyMC3 with a static graph backend (theano), we are able to write things in a declarative way and use the static computational graph to keep track of the dependences. This means we are able to do `log_prob = x.log_prob(x) + y.log_prob(y)`, as the value representation of the random variables `x` and `y` are "swap-in" with some input at runtime. With a dynamic graph we run into problems as we write the model in a forward mode -- we essentially lose track of the dependence in the reverse calling mode. We need to either keep track of the graph representation ourselves, or write a function that could be reevaluated to make sure we have the right dependencies. 

This ideally should look like this

In [66]:
def model(x):
    scale = tfd.HalfCauchy(loc=0, scale=1)
    coefs = tfd.Normal(loc=tf.zeros(x.shape[1]), scale=1)
    predictions = tfd.Normal(loc=tf.tensordot(x, coefs, axes=1), scale=scale)
    return predictions

In [67]:
model(tf.random.normal((100, 10)))

ValueError: TypeError: object of type 'Normal' has no len()


But this function will not work (you can try it yourself), because there is no random variable concept in `tfp`, meaning that you cannot do `RV_a.log_prob(RV_a)` (yes, just think of Random Variable in a PPL as a tensor/array-like object that you can do computation and a log_prob method that we can evaluate it on itself.

In [177]:
def model_w_sample(x):
    scale = tfd.HalfCauchy(loc=0, scale=1).sample()
    coefs = tfd.Normal(loc=tf.zeros(x.shape[1]), scale=1).sample()
    predictions = tfd.Normal(loc=tf.tensordot(x, coefs, axes=1), scale=scale)
    return predictions

In [178]:
model_w_sample(tf.random.normal((100, 10)))

<tfp.distributions.Normal 'Normal' batch_shape=[100] event_shape=[] dtype=float32>

Generating a log_prob from this model also wont work, because the computation is different than the function we have written down above.

What we want here is to track function evaluation at runtime, depending on the context (goal 1 or goal 2 from above).

The very first way to cope with is was writing a wrapper over a distribution object. This wrapper was intended to catch a call to the distribution and use context to figure out what to do: for goal 1, we draw a sample from a random variable and plug the concrete value into the downstream dependencies; for goal 2 we got the concrete value and evaluate it with the respective random variable, and also plug the concrete value into the downstream dependencies. Here we use Coroutines from Python to have dynamic control flow that could achieve this kind of deferred execution.

In [179]:
def model_w_yield(x):
    scale = yield tfd.HalfCauchy(loc=0, scale=1)
    coefs = yield tfd.Normal(loc=tf.zeros(x.shape[1]), scale=1)
    predictions = yield tfd.Normal(loc=tf.tensordot(x, coefs, axes=1), scale=scale)
    return predictions

In [181]:
model_w_yield(tf.random.normal((100, 10)))

<generator object model_w_yield at 0x13dcd2f50>

Now, we evaluate the model as expected but `yield` allows us to defer the program execution. But before evaluating this function, let's figure out what does yield do.

## Sidenote -- Primer on Generators

In [149]:
def generator(message):
    print("I will yield:", message)
    response = yield message
    print("I have:", response)
    return "Nothing to yield, Goodbye!"

In [150]:
g = generator("(generators are cool)")
g

<generator object generator at 0x12ec4e650>

In [151]:
mes = g.send(None)

I will yield: (generators are cool)


In [152]:
mes

'(generators are cool)'

In [153]:
g.send("(Indeed, generators are cool)") 

I have: (Indeed, generators are cool)


StopIteration: Nothing to yield, Goodbye!

#### What has happened here:

* we had a simple generator and were able to communicate with it via `send`
* after `send` is called (first time requires it to have `None` argument) generator goes to the next `yield` expression and yields what it is asked to yield.
* as a return value from `send` we have this exact message from `yield message`
* we set the lhs of `response = yield message` with next `send` and no earlier
* after generator has no `yield` statements left and finally reaches `return`, it raises `StopIteration` with return value as a first argument

Now we are ready to evaluate our model by hand

#### Experiments:

1. Call `next(g)` instead of `g.send(None)`
2. Replace the `return` with `yield`

## Solving the Problem using Generators

In [183]:
state = dict(dists=dict(), samples=dict())
state

{'dists': {}, 'samples': {}}

In [184]:
state["input"] = tf.random.normal((3, 10))
m = model_w_yield(state["input"])

In [185]:
scale_dist = next(m)

In [186]:
print(scale_dist)

tfp.distributions.HalfCauchy("HalfCauchy", batch_shape=[], event_shape=[], dtype=float32)


which means, we are here

```python
def model(x):
    scale = yield tfd.HalfCauchy(0, 1) # <--- HERE
    coefs = yield tfd.Normal(tf.zeros(x.shape[1]), 1, )
    ...
```

What can we do with this distribution? We can choose forward sampling (in this case we sample from the state-less distribution `HalfCauchy(0, 1)`). But we need it to be used by user seamlessly later on regardless of the context (goal 1 or 2 above). On the model side, we need to store intermediate values and its associated distributions (hey! that's a random variable!).

In [187]:
assert scale_dist.name not in state["dists"]
state["samples"][scale_dist.name] = scale_dist.sample()
state["dists"][scale_dist.name] = scale_dist

In [191]:
state

{'dists': {'HalfCauchy': <tfp.distributions.HalfCauchy 'HalfCauchy' batch_shape=[] event_shape=[] dtype=float32>},
 'samples': {'HalfCauchy': <tf.Tensor: shape=(), dtype=float32, numpy=3.615482>},
 'input': <tf.Tensor: shape=(3, 10), dtype=float32, numpy=
 array([[ 1.582552  , -0.06922732, -2.6374202 ,  0.43550617,  0.57397664,
         -0.96478957,  0.1666102 ,  0.6064285 ,  0.24837077,  0.10600287],
        [-0.04085249,  0.9402219 , -1.281183  ,  0.2903516 , -0.18772306,
          0.29066348,  0.7041369 , -0.21032359,  1.1873466 ,  0.06198721],
        [ 0.17795442,  1.0926946 , -0.9444791 , -0.13517664,  0.5391453 ,
         -1.3301303 ,  1.4234358 , -1.2429249 , -1.0964781 , -1.182612  ]],
       dtype=float32)>}

In [192]:
coefs_dist = m.send(state["samples"][scale_dist.name])

In [193]:
print(coefs_dist)

tfp.distributions.Normal("Normal", batch_shape=[10], event_shape=[], dtype=float32)


```python
def model(x):
    scale = yield tfd.HalfCauchy(0, 1)
    coefs = yield tfd.Normal(tf.zeros(x.shape[1]), 1, ) # <--- WE ARE NOW HERE
    predictions = yield tfd.Normal(tf.linalg.matvec(x, coefs), scale)
    return predictions
```

We do the same thing

In [194]:
assert coefs_dist.name not in state["dists"]
state["samples"][coefs_dist.name] = coefs_dist.sample()
state["dists"][coefs_dist.name] = coefs_dist

In [195]:
preds_dist = m.send(state["samples"][coefs_dist.name]) # shouldn't we send state["input"] for x ??

In [196]:
print(preds_dist)

tfp.distributions.Normal("Normal", batch_shape=[3], event_shape=[], dtype=float32)


```python
def model(x):
    scale = yield tfd.HalfCauchy(0, 1)
    coefs = yield tfd.Normal(tf.zeros(x.shape[1]), 1, )
    predictions = yield tfd.Normal(tf.linalg.matvec(x, coefs), scale) # <--- NOW HERE
    return predictions
```

We are now facing predictive distribution. Here we have several options:
* sample from it: we get prior predictive
* set custom values instead of sample, essentially conditioning on data. We might be interested in this to compute unnormalized posterior
* replace it with another distribution, arbitrary magic

In [197]:
assert preds_dist.name not in state["dists"]
state["samples"][preds_dist.name] = tf.zeros(preds_dist.batch_shape)
state["dists"][preds_dist.name] = preds_dist

AssertionError: 

Gotcha, we found duplicated names in our toy graphical model. We can easily tell our user to rewrite the model to get rid of duplicate names

In [198]:
m.throw(RuntimeError(
    "We found duplicate names in your cool model: {}, "
    "so far we have other variables in the model, {}".format(
        preds_dist.name, set(state["dists"].keys()), 
    )
))

RuntimeError: We found duplicate names in your cool model: Normal, so far we have other variables in the model, {'HalfCauchy', 'Normal'}

The good thing is that we *communicate* with user, and can give meaningful exceptions with few pain.

The correct model should look like this:

```python
def model(x):
    scale = yield tfd.HalfCauchy(0, 1)
    coefs = yield tfd.Normal(tf.zeros(x.shape[1]), 1, )
    predictions = yield tfd.Normal(tf.linalg.matvec(x, coefs), scale, name="Normal_1") # <--- HERE we asked out user to change the name
    return predictions
```


Let's set all the names according to the new model and interact with user again using the same model

In [202]:
m.gi_running

False

Our generator is now at the end of its execution - we can't interact with it any more. Let's create a new one and reevaluate with same sampled values (A hint how to get the desired `logp` function)

In [203]:
def model(x):
    scale = yield tfd.HalfCauchy(0, 1)
    coefs = yield tfd.Normal(tf.zeros(x.shape[1]), 1, )
    predictions = yield tfd.Normal(tf.linalg.matvec(x, coefs), scale, name="Normal_1") # <--- HERE we asked out user to change the name
    return predictions

In [204]:
m = model(state["input"])
print(m.send(None))
print(m.send(state["samples"]["HalfCauchy"]))
print(m.send(state["samples"]["Normal"]))
try:
    m.send(tf.zeros(state["input"].shape[0]))
except StopIteration as e:
    stop_iteration = e
else:
    raise RuntimeError("No exception met")

tfp.distributions.HalfCauchy("HalfCauchy", batch_shape=[], event_shape=[], dtype=float32)
tfp.distributions.Normal("Normal", batch_shape=[10], event_shape=[], dtype=float32)
tfp.distributions.Normal("Normal_1", batch_shape=[3], event_shape=[], dtype=float32)


In [205]:
print(stop_iteration)

tf.Tensor([0. 0. 0.], shape=(3,), dtype=float32)


Instead of returning some value in the last `send`, generator raises `StopIteration` because it is exhausted and reached the `return` statement (no more `yield` met). As explained (and checked here) in [PEP0342](https://www.python.org/dev/peps/pep-0342/), we have a return value inside

## Automate the process above

We all are lazy humans and cant stand doing repetitive things. In our model evaluation we followed pretty simple rules:
* asserting name is not used
* checking if we should sample or place a specific value instead
* recording distributions and samples

Next step is to make a function that does all this instead of us. In this tutorial let's keep it simple:

In [ ]:
def interact(gen, state):
    control_flow = gen()
    return_value = None
    while True:
        try:
            dist = control_flow.send(return_value)
            if dist.name in state["dists"]:
                control_flow.throw(RuntimeError(
                    "We found duplicate names in your cool model: {}, "
                    "so far we have other variables in the model, {}".format(
                        dist.name, set(state["dists"].keys()), 
                    )
                ))
            if dist.name in state["samples"]:
                return_value = state["samples"][dist.name]
            else:
                return_value = dist.sample()
                state["samples"][dist.name] = return_value
            state["dists"][dist.name] = dist
        except StopIteration as e:
            if e.args:
                return_value = e.args[0]
            else:
                return_value = None
            break
    return return_value, state

This implementation assumes no arg generator, we make things just simple

In [ ]:
preds, state = interact(lambda: model(tf.random.normal((3, 10))), state=dict(dists=dict(), samples=dict()))

In [ ]:
state

In [ ]:
preds

We get all the things as expected. To calculate `logp` you just iterate over distributions and match them with the correspondig values. But let's dive deeper

## One level deeper

Recall the motivating example from [PR#125](https://github.com/pymc-devs/pymc4/pull/125)

In [ ]:
def Horseshoe(mu=0, tau=1., s=1., name=None):
    with tf.name_scope(name):
        scale = yield tfd.HalfCauchy(0, s, name="scale")
        noise = yield tfd.Normal(0, tau, name="noise")
        return scale * noise + mu


def linreg(x):
    scale = yield tfd.HalfCauchy(0, 1, name="scale")
    coefs = yield Horseshoe(tf.zeros(x.shape[1]), name="coefs")
    predictions = yield tfd.Normal(tf.linalg.matvec(x, coefs), scale, name="predictions")
    return predictions

In [ ]:
preds, state = interact(lambda: linreg(tf.random.normal((3, 10))), state=dict(dists=dict(), samples=dict()))

Oooups, we have a type error. What we want is a nested model, but nesting models is something different from a plain generator. As we have our model being a generator itself, the return value of `Horseshoe(tf.zeros(x.shape[1]), name="coefs")` is a generator. Of course this generator has no name attribute. Okay, we can ask user to use `yield from` construction to generate from the generator

In [ ]:
def linreg_ugly(x):
    scale = yield tfd.HalfCauchy(0, 1, name="scale")
    coefs = yield from Horseshoe(tf.zeros(x.shape[1]), name="coefs")
    predictions = yield tfd.Normal(tf.linalg.matvec(x, coefs), scale, name="predictions")
    return predictions

In [ ]:
preds, state = interact(lambda: linreg_ugly(tf.random.normal((3, 10))), state=dict(dists=dict(), samples=dict()))

Okay, we passed this thing

In [ ]:
state["dists"]

We got nesting models working, but it requires `yield from`. This is UGLY and potentially confusing for user. Fortunately, we can rewrite out `interact` function to accept nested models in a few lines, and let the Python do the task for us.

In [ ]:
import types
def interact_nested(gen, state):
    # for now we should check input type
    if not isinstance(gen, types.GeneratorType):
        control_flow = gen()
    else:
        control_flow = gen

    return_value = None
    while True:
        try:
            dist = control_flow.send(return_value)
            # this makes nested models possible
            if isinstance(dist, types.GeneratorType):
                return_value, state = interact_nested(dist, state)
                # ^ in a few lines of code, go recursive
            else:
                if dist.name in state["dists"]:
                    control_flow.throw(RuntimeError(
                        "We found duplicate names in your cool model: {}, "
                        "so far we have other variables in the model, {}".format(
                            dist.name, set(state["dists"].keys()), 
                        )
                    ))
                if dist.name in state["samples"]:
                    return_value = state["samples"][dist.name]
                else:
                    return_value = dist.sample()
                    state["samples"][dist.name] = return_value
                state["dists"][dist.name] = dist
        except StopIteration as e:
            if e.args:
                return_value = e.args[0]
            else:
                return_value = None
            break
    return return_value, state

remember we had problems here:
```python
preds, state = interact(lambda: linreg(tf.random.normal((3, 10))), state=dict(dists=dict(), samples=dict()))
```

Additionally we can specify the observed variable

In [ ]:
preds, state = interact_nested(lambda: linreg(tf.random.normal((3, 10))), state=dict(dists=dict(), samples={"predictions/":tf.zeros(3)}))

In [ ]:
state["dists"]

In [ ]:
state["samples"]

Cool, we've finished the central idea behind PyMC4 core engine. There is some extra stuff to do to make `evaluate_nested` really powerful

* resolve transforms
* resolve reparametrizations
* variational inference
* better error messages
* lazy returns in posterior predictive mode

Some of this functionality may be found in the corresponding [PR#125](https://github.com/pymc-devs/pymc4/pull/125)